In [187]:
import numpy as np
import pandas as pd
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [188]:
lines_train = []
lines_test = []
classes = []
categories = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

with open("data/train.dat", "r") as fh:
    lines = fh.readlines()
    for l in lines:
        lines_train.append(l.rstrip().split(' '))
    
with open("data/train.labels", "r") as fh:
    lines = fh.readlines()
    for l in lines:
        classes.append(l.rstrip())
    
with open("data/test.dat", "r") as fh:
    lines = fh.readlines()
    for l in lines:
        lines_test.append(l.rstrip().split(' '))

In [189]:
print(len(lines_train[0]))
print(len(lines_test))
print(len(classes))


887
5296
21186


In [190]:
# print(lines_train[:1])
# print(lines_test[:1])
# print(classes[:1])

In [191]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from time import time

In [192]:
from sklearn.preprocessing import StandardScaler
X_train_std = StandardScaler().fit_transform(lines_train)
X_test_std = StandardScaler().fit_transform(lines_test)
# vectorizer = HashingVectorizer()
# X_train_std = vectorizer.transform(lines_train)
# X_test_std = vectorizer.transform(lines_test)

In [193]:
print(type(X_train_std))
print(X_train_std.shape)
print(type(X_test_std))
print(X_test_std.shape)

<type 'numpy.ndarray'>
(21186, 887)
<type 'numpy.ndarray'>
(5296, 887)


In [194]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=48)
# pca.fit(X_train_std)
# X_train_svd = pca.transform(X_train_std)
# X_test_svd = pca.transform(X_test_std)

In [195]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=48, n_iter=10, random_state=42)
svd.fit(X_train_std)
X_train_svd = svd.transform(X_train_std)
X_test_svd = svd.transform(X_test_std)

In [196]:
print(X_train_svd)
print(X_test_svd)

[[  2.86705424e+01   1.72106524e+00  -2.50534664e-01 ...,  -2.38275111e-15
    4.87457297e-15  -9.92261828e-16]
 [  2.86705424e+01   3.85705164e+00   4.36661196e+00 ...,  -2.40064045e-15
    5.39152056e-15  -9.02056208e-16]
 [  2.86705424e+01  -9.69232395e+00  -3.26048524e+00 ...,  -2.02214547e-15
    4.48252546e-15  -1.08246745e-15]
 ..., 
 [  2.86705424e+01  -3.83621228e+00  -4.72358943e+00 ...,  -3.05712487e-15
    4.02108902e-15  -2.76861867e-15]
 [  2.86705424e+01  -1.08585838e+01  -2.17295332e+00 ...,  -2.87866519e-15
    3.08086889e-15  -2.72351586e-15]
 [  2.86705424e+01  -2.02996547e+00  -4.83510568e+00 ...,  -2.95759511e-15
    4.13211132e-15  -3.02188830e-15]]
[[-1.04637016  4.58987522 -2.39569287 ...,  2.04552982 -1.33351615
  -0.05464101]
 [-1.04637016 -8.97980057 -0.52335762 ...,  2.04552982 -1.33351615
  -0.05464101]
 [-1.04637016 -7.05852548  1.29715189 ...,  2.04552982 -1.33351615
  -0.05464101]
 ..., 
 [-1.04637016  1.77707858 -0.67312864 ...,  2.04552982 -1.33351615


In [197]:
# vectorizer = HashingVectorizer(stop_words='english')
# X_train = vectorizer.transform(lines_train)
# X_test = vectorizer.transform(lines_test)
# vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
# X_train = vectorizer.fit_transform(docs_train)


y_train = np.asarray(classes)


In [198]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(k_neighbors=2, ratio="minority")
X_resampled, y_resampled = smote.fit_sample(X_train_svd, y_train)
# X_res_vis = pca.transform(X_resampled)
print(type(X_resampled))

<type 'numpy.ndarray'>


In [199]:
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(ratio="minority")
# X_resampled, y_resampled = ros.fit_sample(X_train_pca, y_train)

In [200]:
# from imblearn.over_sampling import ADASYN 
# ada = ADASYN(random_state=10, n_neighbors=2)
# X_resampled, y_resampled = ada.fit_sample(X_train_pca, y_train)



In [201]:
print(X_resampled.shape)
print(y_resampled.shape)

(31558, 48)
(31558,)


In [202]:
# target_names = ["1", "2", "3", "4", "5"]
def classify(clf, X_train, Y_train, X_test):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, Y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    print(type(pred))
    return pred

In [203]:
# pred1 = classify(KNeighborsClassifier(n_neighbors=4), X_train_pca, y_train, X_test_pca)
pred1 = classify(KNeighborsClassifier(n_neighbors=4), X_resampled, y_resampled, X_test_svd)
# pred1 = classify(SVC(gamma=2), X_resampled, y_resampled, X_test_pca)
# pred1 = classify(AdaBoostClassifier(), X_resampled, y_resampled, X_test_pca)

________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')
train time: 0.107s
test time:  5.405s
<type 'numpy.ndarray'>


In [204]:
def writeToFile(pred):
    with open('output/output_KNeighborSVD48.dat','w+') as f:
        for p in pred:
            f.write(str(p)+"\n")

In [205]:
print(type(pred1))
writeToFile(pred1)

<type 'numpy.ndarray'>


In [206]:
def readFile(filename):
    content = []
    with open(filename) as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
    return content

In [207]:
def compare(p1, p2):
    match = 0
    total = len(p1)
    for i in range(0,total-1):                
        if(p1[i]==p2[i]):
            match = match+1
    per = ((float(match)/float(total))*100)
    return per
    

In [208]:
# p1 = readFile("output/output_KNeighbor1.dat")
p2 = readFile("submissions/Submission2_0.8168/output_KNeighbor.dat")
print(compare(pred1,p2))
print(compare(pred1,readFile("submissions/Submission11_0.81/output_KNeighbor.dat")))


84.4410876133
99.3013595166


In [209]:
print(pred1[:10])
print(p2[:10])
print(len(pred1))
print(len(p2))

['1' '1' '2' '2' '1' '2' '1' '1' '1' '1']
['1', '1', '2', '2', '2', '2', '1', '1', '1', '1']
5296
5296
